In [ ]:
# Install necessary libraries
!pip install autoevals duckdb braintrust openai transformers arabic-reshaper python-bidi --quiet
!pip install nest-asyncio
!pip install --upgrade openai


In [ ]:
import os
from openai import AsyncOpenAI
import braintrust
import asyncio
import json
import random
from dataclasses import dataclass
from typing import List, Dict, Any
import arabic_reshaper
from bidi.algorithm import get_display
import autoevals


In [ ]:
# Set  API keys
os.environ["OPENAI_API_KEY"] = ""
os.environ["BRAINTRUST_API_KEY"] = ""

# Log in to Braintrust
braintrust.login(api_key=os.environ["BRAINTRUST_API_KEY"],force_login=True)
client = braintrust.wrap_openai(AsyncOpenAI(api_key=os.environ["OPENAI_API_KEY"]))


In [ ]:
import json
from dataclasses import dataclass
from typing import Dict, Any, List
import arabic_reshaper
from bidi.algorithm import get_display

@dataclass
class ArabicQuestionAnswer:
    passage: str
    question: str
    expected_answer: str
    generated_answer: str = ""
    metadata: Dict[str, Any] = None

    # Function to load Arabic QA dataset

def load_dataset(file_path: str) -> List[ArabicQuestionAnswer]:
    """Load and parse the Arabic QA dataset from a JSON file."""
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        qa_pairs = []
        for entry in data['data']:
            for paragraph in entry['paragraphs']:
                context = paragraph['context']
                for qa in paragraph['qas']:
                    if qa['answers']:  # Ensure there are answers
                        qa_pairs.append(
                            ArabicQuestionAnswer(
                                 passage=context,
                                question=qa['question'],
                                expected_answer=qa['answers'][0]['text']
                            )
                        )
        return qa_pairs

# Load Arabic QA dataset (update the file path based on  Colab setup)
qa_pairs = load_dataset('/content/train.json')  # Replace with your file path

print(f"Loaded {len(qa_pairs)} QA pairs.")
for qa in qa_pairs[:5]:  # Display the first 5 pairs

    print(f"Passage: {qa.passage}")
    print(f"Question: {qa.question}")
    print(f"Expected Answer: {qa.expected_answer}")
    print("-" * 50)
    print(len(qa_pairs))


Loaded 64782 QA pairs.
Passage: إستونيا ، رسميًا جمهورية إستونيا ؛ هي دولة تقع في منطقة بحر البلطيق بشمال أوروبا. يحدها من الشمال خليج فنلندا، ومن الغرب بحر البلطيق، ومن الجنوب لاتفيا (343 كم)، وإلى الشرق من بحيرة بيبوس والاتحاد الروسي (338.6 كم). تغطي أراضي إستونيا ما مساحته 45227 كيلومتر مربع (17462) ميل مربع، ويعدّ مناخها الموسمي معتدلًا. الإستونية هي اللغة الرسمية الوحيدة في الدولة. تٌعتبر إستونيا جمهورية ديمقراطية برلمانية، مقسمة إلى 15 مقاطعة. العاصمة وأكبر مدنها العاصمة تالين. ويبلغ عدد سكانها 1,319,133مليون نسمة. كما تعدّ واحدة من الأعضاء الأقل نموا من حيث عدد السكان في الاتحاد الأوروبي، ومنطقة اليورو، ومنظمة حلف شمال الأطلسي. ولديها أعلى الناتج المحلي الإجمالي للفرد الواحد بين جمهوريات الاتحاد السوفيتي السابق. تم وصف جمهورية إستونيا بأنها «اقتصاد ذو دخل مرتفع» من قبل البنك الدولي وبأنها «اقتصاد متطور» من قبل صندوق النقد الدولي، وقد أصبحت في وقت لاحق عضوًا في منظمة التعاون الاقتصادي والتنمية. تصنف الأمم المتحدة إستونيا كدولة متقدمة، كما يعدّ مؤشر التنمية البشرية عاليًا جدًا. وك

In [ ]:
random.seed(42)  # Ensures reproducibility of any randomized operations

async def hallucinate_answer(qa):
    # Function to generate hallucinated answers using OpenAI API.
    response = await client.chat.completions.create(
        model="gpt-3.5-turbo",  # Model name
        messages=[
            {
                "role": "system",
                "content": """\
أنت مساعد هلوسة مفيد، يختلق إجابات وهمية للأسئلة.

أجب عن السؤال التالي في جملة واحدة. إذا كنت تعرف الإجابة، فابتكر بعض التفاصيل الغير صحيحه و التي ليست في النص الذي حفظته.

تأكد دائمًا من الإجابة بثقة، حتى لو لم تكن تعرف الإجابة. لا تستخدم كلمات مثل "ربما"، "على الأرجح"، "قد"، أو أي علامات ترقيم مثل "...". لا تعترف أنك لا تستطيع أو لا تعرف الإجابة."""
            },
            {
                "role": "user",
                "content": qa.question  # The user's question from the QA pair.
            }
        ],
        temperature=1,
        max_tokens=150,  # Limit the response length for testing.
    )
    return response.choices[0].message.content

# Use a subset of qa_pairs for faster testing.
sample_qa_pairs = qa_pairs[:50]  # Change this number to adjust the size of the dataset used in testing.

# Asynchronously generate hallucinated answers
hallucinated_answers = await asyncio.gather(
    *[hallucinate_answer(qa) for qa in sample_qa_pairs]
)

# Filter and create hallucination objects based on specific criteria.
hallucinations = [
    ArabicQuestionAnswer(
        passage=qa.passage,
        question=qa.question,
        expected_answer=qa.expected_answer,
        generated_answer=hallucination,
    )
    for (qa, hallucination) in zip(sample_qa_pairs, hallucinated_answers)
    # Exclude simple yes/no answers.
    if "yes" not in hallucination.lower() and "no" not in hallucination.lower()
]

# Print the first hallucination for verification.
print("Passage:")
print(hallucinations[0].passage)
print("\nQuestion:")
print(hallucinations[0].question)
print("\nExpected Answer:")
print(hallucinations[0].expected_answer)
print("\nGenerated Answer:")
print(hallucinations[0].generated_answer)

# Display the total number of valid hallucinations.
print("\n\nNumber of hallucinations:", len(hallucinations))


Passage:
إستونيا ، رسميًا جمهورية إستونيا ؛ هي دولة تقع في منطقة بحر البلطيق بشمال أوروبا. يحدها من الشمال خليج فنلندا، ومن الغرب بحر البلطيق، ومن الجنوب لاتفيا (343 كم)، وإلى الشرق من بحيرة بيبوس والاتحاد الروسي (338.6 كم). تغطي أراضي إستونيا ما مساحته 45227 كيلومتر مربع (17462) ميل مربع، ويعدّ مناخها الموسمي معتدلًا. الإستونية هي اللغة الرسمية الوحيدة في الدولة. تٌعتبر إستونيا جمهورية ديمقراطية برلمانية، مقسمة إلى 15 مقاطعة. العاصمة وأكبر مدنها العاصمة تالين. ويبلغ عدد سكانها 1,319,133مليون نسمة. كما تعدّ واحدة من الأعضاء الأقل نموا من حيث عدد السكان في الاتحاد الأوروبي، ومنطقة اليورو، ومنظمة حلف شمال الأطلسي. ولديها أعلى الناتج المحلي الإجمالي للفرد الواحد بين جمهوريات الاتحاد السوفيتي السابق. تم وصف جمهورية إستونيا بأنها «اقتصاد ذو دخل مرتفع» من قبل البنك الدولي وبأنها «اقتصاد متطور» من قبل صندوق النقد الدولي، وقد أصبحت في وقت لاحق عضوًا في منظمة التعاون الاقتصادي والتنمية. تصنف الأمم المتحدة إستونيا كدولة متقدمة، كما يعدّ مؤشر التنمية البشرية عاليًا جدًا. وكذلك أنها تتمتع بمستوى ع

In [ ]:
PROMPT = """\
أنت تقوم بمقارنة إجابة مقدمة مع إجابة خبير لسؤال معين. فيما يلي البيانات:
[BEGIN DATA]
************
[Question]: {input}
************
[Expert]: {expected}
************
[Submission]: {output}
************
[END DATA]

قارن المحتوى العلمي للإجابة المقدمة مع إجابة الخبير. تجاهل أي اختلافات في الأسلوب، القواعد، أو علامات الترقيم.
قم بتقييم الإجابة المقدمة على مقياس من 1 إلى 10.
"""

In [ ]:

@braintrust.traced
async def numeric_rater(input, output, expected):
    response = await client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": PROMPT.format(input=input, output=output, expected=expected)
            }
        ],
        temperature=0,
        tools=[
            {
                "type": "function",
                "function": {
                    "name": "rate",
                    "description": "قم بتقييم الإجابة المقدمة على مقياس من 1 إلى 10.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "rating": {"type": "integer", "minimum": 1, "maximum": 10},
                        },
                        "required": ["rating"],
                    },
                },
            }
        ],
        tool_choice={"type": "function", "function": {"name": "rate"}},
        )

    arguments = json.loads(response.choices[0].message.tool_calls[0].function.arguments)
    return (arguments["rating"] - 1) / 9

print(qa_pairs[10].question,
      "On a correct answer:",
    qa_pairs[10].expected_answer)
print(
    await numeric_rater(
        qa_pairs[10].question,
        qa_pairs[10].generated_answer,
        qa_pairs[10].expected_answer,
    )
)

print(
    hallucinations[10].question,
    "On a hallucinated answer:",
    hallucinations[10].generated_answer,
)
print(
    await numeric_rater(
        hallucinations[10].question,
        hallucinations[10].generated_answer,
        hallucinations[10].expected_answer,
    )
)

ما هي نسبة السكان في إستونيا الذين يعتبرون أن الدين ليس له علاقة بحياتهم؟ On a correct answer: ثلثي الشعب
8
ما هي نسبة السكان في إستونيا الذين يعتبرون أن الدين ليس له علاقة بحياتهم؟ On a hallucinated answer: نسبة السكان في إستونيا الذين يعتبرون أن الدين ليس له علاقة بحياتهم تبلغ حوالي 30%، وهم يفضلون التركيز على الجوانب العلمانية في الحياة.
7


In [ ]:
from dataclasses import asdict

from braintrust import Eval


def data():
    for pair in hallucinations:
        yield dict(
            input=dict(asdict(pair)), expected=0, metadata=dict(hallucination=True)
        )


async def task(input):
    return await numeric_rater(
        input=input["question"],
        output=input["generated_answer"],
        expected=input["expected_answer"],
    )


def normalized_diff(output, expected):
    return 1 - abs(output - expected)


await Eval(
    "LLM-as-a-judge",
    data=data,
    task=task,
    scores=[normalized_diff],
    experiment_name="Numeric rater",
    max_concurrency=10,
)

Experiment Numeric rater-5d902486 is running at https://www.braintrust.dev/app/king%20saud/p/LLM-as-a-judge/experiments/Numeric%20rater-5d902486
LLM-as-a-judge [experiment_name=Numeric rater] (data): 100it [00:00, 10931.21it/s]


LLM-as-a-judge [experiment_name=Numeric rater] (tasks):   0%|          | 0/100 [00:00<?, ?it/s]

log request failed. Elapsed time: 0.6981720924377441 seconds. Payload size: 1621602.
Error: 400: {"Code":"BadRequestError","Message":"You have hit your free tier limit. Please contact us at info@braintrustdata.com to discuss pricing.\n\nFull debug details:\nViolations of resource constraint num_private_experiment_row_actions: {\"(210c3dcd-cf21-4101-befc-a4632fe27ffa,1022,1000)\"} [user_email=altwujaeire.reema@gmail.com] [user_org=king saud] [timestamp=1732067200.746] [user_email=altwujaeire.reema@gmail.com] [user_org=king saud] [timestamp=1732067201.046]"}
log request failed. Elapsed time: 0.5217251777648926 seconds. Payload size: 1621602.
Error: 400: {"Code":"BadRequestError","Message":"You have hit your free tier limit. Please contact us at info@braintrustdata.com to discuss pricing.\n\nFull debug details:\nViolations of resource constraint num_private_experiment_row_actions: {\"(210c3dcd-cf21-4101-befc-a4632fe27ffa,1022,1000)\"} [user_email=altwujaeire.reema@gmail.com] [user_org=kin

EvalResultWithSummary(summary="...", results=[...])

In [ ]:
from autoevals import LLMClassifier


In [ ]:

# Define your custom prompt for comparison
PROMPT = """\
أنت تقوم بمقارنة إجابة مقدمة مع إجابة خبير لسؤال معين. فيما يلي البيانات:
[BEGIN DATA]
************
[Question]: {{input}}
************
[Expert]: {{expected}}
************
[Submission]: {{output}}
************
[END DATA]

قارن المحتوى العلمي للإجابة المقدمة مع إجابة الخبير. تجاهل أي اختلافات في الأسلوب، القواعد، أو علامات الترقيم.
قد تكون الإجابة المقدمة جزءًا من إجابة الخبير أو إضافة لها، أو قد تتعارض معها. حدد أي حالة تنطبق. إجابة السؤال تكون على النحو التالي:
(A) الإجابة المقدمة هي جزء من إجابة الخبير ومتوافقة تمامًا معها.
(B) الإجابة المقدمة هي إضافة لإجابة الخبير ومتوافقة تمامًا معها.
(C) الإجابة المقدمة تحتوي على نفس التفاصيل مثل إجابة الخبير.
(D) هناك اختلاف بين الإجابة المقدمة وإجابة الخبير.
(E) الإجابات تختلف، لكن هذه الاختلافات غير هامة من حيث المحتوى العلمي.

قم بالإجابة بعد اتخاذ قرار باستخدام `select_choice` مع تفسيرك خطوة بخطوة للتأكد من صحة استنتاجك. تجنب مجرد بيان الإجابة الصحيحة في البداية. اختر إجابة واحدة بتحديد المعامل `choice` مع الإجابة من A، B، C، D، أو E.
"""


In [ ]:
# Since we're testing for hallucinations, penalize (B) as much as (D).
CHOICE_SCORES = {
    "A": 0.5,
    "B": 0,
    "C": 1,
    "D": 0,
    "E": 1,
}

@braintrust.traced
async def classifier(input, output, expected):
    response = await client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": PROMPT.format(input=input, output=output, expected=expected),
            }
        ],
        temperature=0,
        tools=[
            {
                "type": "function",
                "function": {
                    "name": "rate",
                    "description": "Call this function to select a choice.",
                    "parameters": {
                        "properties": {
                            "reasons": {
                                "description": "Write out in a step by step manner your reasoning to be sure that your conclusion is correct. Avoid simply stating the correct answer at the outset.",
                                "type": "string",
                            },
                            "choice": {
                                "description": "The choice",
                                "type": "string",
                                "enum": ["A", "B", "C", "D", "E"],
                            },
                        },
                        "required": ["reasons", "choice"],
                        "type": "object",
                    },
                },
            }
        ],
        tool_choice={"type": "function", "function": {"name": "rate"}},
    )
    arguments = json.loads(response.choices[0].message.tool_calls[0].function.arguments)
    choice = arguments["choice"]
    return CHOICE_SCORES[choice] if choice in CHOICE_SCORES else None


print(qa_pairs[10].question, "On a correct answer:", qa_pairs[10].generated_answer)
print(
    await classifier(
        qa_pairs[10].question,
        qa_pairs[10].generated_answer,
        qa_pairs[10].expected_answer,
    )
)

print(
    hallucinations[10].question,
    "On a hallucinated answer:",
    hallucinations[10].generated_answer,
)
print(
    await classifier(
        hallucinations[10].question,
        hallucinations[10].generated_answer,
        hallucinations[10].expected_answer,
    )
)

ما هي نسبة السكان في إستونيا الذين يعتبرون أن الدين ليس له علاقة بحياتهم؟ On a correct answer: 
0.5
ما هي نسبة السكان في إستونيا الذين يعتبرون أن الدين ليس له علاقة بحياتهم؟ On a hallucinated answer: نسبة السكان في إستونيا الذين يعتبرون أن الدين ليس له علاقة بحياتهم تبلغ حوالي 30%، وهم يفضلون التركيز على الجوانب العلمانية في الحياة.
1


In [ ]:
async def task(input):
    return await classifier(
        input=input["question"],
        output=input["generated_answer"],
        expected=input["expected_answer"],
    )


await Eval(
    "LLM-as-a-judge",
    data=data,
    task=task,
    scores=[normalized_diff],
    experiment_name="Classifier",
    max_concurrency=10,
)

Experiment Classifier is running at https://www.braintrust.dev/app/king%20saud/p/LLM-as-a-judge/experiments/Classifier
LLM-as-a-judge [experiment_name=Classifier] (data): 20it [00:00, 6661.86it/s]


LLM-as-a-judge [experiment_name=Classifier] (tasks):   0%|          | 0/20 [00:00<?, ?it/s]


=========================SUMMARY=========================
Classifier compared to Numeric rater-c38db2b9:
75.00% (+15.56%) 'normalized_diff' score	(12 improvements, 5 regressions)

2.55s (+186.59%) 'duration'         	(0 improvements, 20 regressions)
2.51s (+185.52%) 'llm_duration'     	(0 improvements, 20 regressions)
389tok (+15320.00%) 'prompt_tokens'    	(0 improvements, 20 regressions)
180.15tok (+17515.00%) 'completion_tokens'	(0 improvements, 20 regressions)
569.15tok (+32835.00%) 'total_tokens'     	(0 improvements, 20 regressions)
0.00$ (+00.21%) 'estimated_cost'   	(0 improvements, 20 regressions)

See results for Classifier at https://www.braintrust.dev/app/king%20saud/p/LLM-as-a-judge/experiments/Classifier


EvalResultWithSummary(summary="...", results=[...])

In [ ]:
#next do this
PROMPT = """\
You are comparing a submitted answer to an expert answer on a given question. Here is the data:
[BEGIN DATA]
************
[Question]: {{input}}
************
[Expert]: {{expected}}
************
[Submission]: {{output}}
************
[END DATA]

Compare the factual content of the submitted answer with the expert answer. Ignore any differences in style, grammar, or punctuation.
The submitted answer may either be a subset or superset of the expert answer, or it may conflict with it. Determine which case applies. Answer the question by selecting one of the following options:
(A) The submitted answer is a subset of the expert answer and is fully consistent with it.
(B) The submitted answer is a superset of the expert answer and is fully consistent with it.
(C) The submitted answer contains all the same details as the expert answer.
(D) There is a disagreement between the submitted answer and the expert answer.
(E) The answers differ, but these differences don't matter from the perspective of factuality.

Answer the question by calling `select_choice` with your reasoning in a step-by-step matter to be
sure that your conclusion is correct. Avoid simply stating the correct answer at the outset. Select a
single choice by setting the `choice` parameter to a single choice from A, B, C, D, or E.
"""

Classifier = autoevals.LLMClassifier(
    name="Hallucination detector",
    prompt_template=PROMPT,
    choice_scores={"A": 0.5, "B": 0, "C": 1, "D": 0, "E": 1},
    use_cot=True,
)

In [ ]:
async def task(input):
    return await classifier(
        input=input["question"],
        output=input["generated_answer"],
        expected=input["expected_answer"],
    )


await Eval(
    "LLM-as-a-judge",
    data=data,
    task=task,
    scores=[normalized_diff],
    experiment_name="Classifier",
    max_concurrency=10,
)

Experiment Classifier-76373ce0 is running at https://www.braintrust.dev/app/mee/p/LLM-as-a-judge/experiments/Classifier-76373ce0
LLM-as-a-judge [experiment_name=Classifier] (data): 50it [00:00, 11339.02it/s]


LLM-as-a-judge [experiment_name=Classifier] (tasks):   0%|          | 0/50 [00:00<?, ?it/s]

log request failed. Elapsed time: 0.997020959854126 seconds. Payload size: 2934707.
Error: 400: {"Code":"BadRequestError","Message":"You have hit your free tier limit. Please contact us at info@braintrustdata.com to discuss pricing.\n\nFull debug details:\nViolations of resource constraint num_private_experiment_row_actions: {\"(4bf89e04-1679-41fc-bfd8-adf40c8f8126,1017,1000)\"} [user_email=ryryaltwuaijri@gmail.com] [user_org=mee] [timestamp=1732068038.471] [user_email=ryryaltwuaijri@gmail.com] [user_org=mee] [timestamp=1732068038.976]"}
log request failed. Elapsed time: 0.5128757953643799 seconds. Payload size: 2934707.
Error: 400: {"Code":"BadRequestError","Message":"You have hit your free tier limit. Please contact us at info@braintrustdata.com to discuss pricing.\n\nFull debug details:\nViolations of resource constraint num_private_experiment_row_actions: {\"(4bf89e04-1679-41fc-bfd8-adf40c8f8126,1017,1000)\"} [user_email=ryryaltwuaijri@gmail.com] [user_org=mee] [timestamp=173206803


=========================SUMMARY=========================
Classifier-76373ce0 compared to Classifier:
100.00% 'normalized_diff' score

2.90s duration
3.43s llm_duration
110.88tok prompt_tokens
82.82tok completion_tokens
193.71tok total_tokens
0.00$ estimated_cost

See results for Classifier-76373ce0 at https://www.braintrust.dev/app/mee/p/LLM-as-a-judge/experiments/Classifier-76373ce0


EvalResultWithSummary(summary="...", results=[...])